
<b> I. HPO metrics <br> </b>
<b> II.Orpha - HPOs<br> </b>
<b> III. Patients - HPO<br> </b>
<b> IV. Count HPOs per patient and HPOs per Orpha<br> </b>
<b> V Build the df frequence of HPOs orphacode <br> </b>

<br>

In [1]:
def get_parent_of_ancestor(str_sentence):
    """
    From : {HPOTerm(id='HP:0011804'", " name='Abnormal muscle physiology'"," is_a=['HP:0003011 ! Abnormality of the musculature'])"," HPOTerm(id='HP:0040064'"," name='Abnormality of limbs'",
    ...
    to :
      {'HP:0000001': 'is_a=[])', 'HP:0011804': 'HP:0003011', 'HP:0040064': 'HP:0000118', 'HP:0000118': 'HP:0000001', 'HP:0003011': 'HP:0033127', 'HP:0033127': 'HP:0000118', 'HP:0001324': 'HP:0011804'}
      
    From str into dict 

    """
    ancertor_dict = {}
    one_split = str(str_sentence).split(',')
    is_a = id = ""
    for one_el in one_split:
        #print(one_el)
        if "HPOTerm" in one_el:
            #print(one_el)
            id = one_el.split("id='")[1].replace("'",'')
        elif "name=" in one_el:
            #print('Name : {}'.format(one_el))
            pass
        elif "is_a=" in one_el:
            #print('is_a : {}'.format(one_el))
            is_a_unstruct = one_el.replace("is_a=['",'')
            is_a = is_a_unstruct.split("!")[0].strip()
        ancertor_dict[id] = is_a
    return ancertor_dict

        

In [2]:

#Least commons ancestor
def get_deepest_ancestor(str_sentence_common_ancestor):
    """ 
        trouver l'ancetre commun le plus profond en evalue chaque clé valeur
        il faut que la clé (enfant) ne se retrouve pas dans la valeur du dict (parent)
        From dict to list
        from :{'HP:0000001': 'is_a=[])', 'HP:0011804': 'HP:0003011', 'HP:0040064': 'HP:0000118', 'HP:0000118': 'HP:0000001', 'HP:0003011': 'HP:0033127', 'HP:0033127': 'HP:0000118', 'HP:0001324': 'HP:0011804'}
        key : chilren, value : ancester
        to :['HP:0040064', 'HP:0001324']
    """
    ancertor_dict = get_parent_of_ancestor(str_sentence_common_ancestor)
    #print("AC dict \n : {}".format(ancertor_dict))
    list_LCAs = []  
    for key in ancertor_dict.keys():
        if key not in list(ancertor_dict.values()):
            #print("one LCA : {}".format(key))
            list_LCAs.append(key)
    return list_LCAs

##############

In [3]:
def extracts_LCAdescendant(hpo_LCA,orphacode_item_dict):
    """ From the LCA found all it descendant"""

    ## Extrait tout les descendants a partir du LCA 

    # extrait tout les LCAs trouver
    all_LCA = []
    all_LCA.append(hpo_LCA)
    #print(all_LCA)

    for oneLCA in all_LCA:
        #print(oneLCA)
        # parcourt l'onthology et extrait les enfants 
        list_child = Ontology.get_hpo_object(oneLCA).children
        # parcourt les enfants
        for one_children in list_child:
            if one_children.id not in all_LCA: # si le descendant n'est pas dans la liste de tout les LCAs
                if orphacode_item_dict[one_children.id] != 0: # si le nombre d'occurence est diff de 0 donc non present dans la base 
                    all_LCA.append(one_children.id) 
                    #print("len child {}\t id:{}\tnb occu: {}".format(len(list_child),one_children.id,orphacode_dict[one_children.id]))
    return all_LCA



In [4]:

def enumerated_LCAdescents_per_disease(all_LCA_desd,orphacode_dict):
    """ From  all hpos LCA end their descendant count the disease where its appear"""
    ## pour le LCA et ses descendants on regarde le nombre de maladies ou ils sont presents 
    orpha_LCAs = set()

    for one_orpha in orphacode_dict.keys(): 
        # parcourt en premier tout les orpha
        for onehpochild in all_LCA_desd:
            # pour chaque orpha on regarde si le LCA et ses descants sont present 
            if onehpochild in orphacode_dict[one_orpha]:
                # si l'hpo est present on incremente la liste de la maladie
                orpha_LCAs.add(one_orpha)
    return orpha_LCAs


#### export in txt frmat

In [5]:
def export_file(path, d_list,str_name):
    with open(path +"\\" + str_name + ".txt", 'a') as f:

        for oneel in d_list:
            # we use ; to split because it's esier to handle 
            f.write(str(oneel) + ";")

### librairy

In [6]:
import json # use to extrat nb of hpo from hpo.json
from pyhpo import stats, Ontology, HPOSet,term # hpo3
import xmltodict # for the orphcode pd4
import collections # for count nb of hpo in patient or orphacode
import os 
import math # for the log to get the IC
import pandas as pd

### Path

In [7]:
path_projet = r"C:\Users\mchahdil\Desktop\compare_study\\"
path_input = path_projet + r"\input_files\\"
path_output = path_projet + r"\output_files\\"


path_phenopacket = r"C:\Users\mchahdil\Desktop\data_patient\json_raw\\"
path_orpha = path_input+r'\orphanet_files'
 

### I. HPO metric 

<b> rsd: 16614 terms</b>

###################

<b> Load hpo3 : </b>

In [8]:
from pyhpo import Ontology
#Ontology(path_hpo+"/ontology/")
Ontology()

i=0
# iterating all HPO terms
for term in Ontology:
    #print(term.name)
    i=i+1
i

17059

### II.Orpha - HPOs <br>
<b>  rsd : 4210 terms </b><br>
We set IC of these to : 8.345217926676428 = -1 * (log(1 / 4210)

In [9]:

orphacode_info=dict()
all_hpo_orphacode = []
with open(path_orpha+'/en_product4.xml') as pd_xml:
    root = xmltodict.parse(pd_xml.read())
xml_dict = root['JDBOR']['HPODisorderSetStatusList']['HPODisorderSetStatus']
for one_dict in xml_dict:
    hpo_list = []
    section_disorder = one_dict['Disorder']
    #print(section_disorder['OrphaCode'])
    section_hpo = section_disorder['HPODisorderAssociationList']
    #print(section_hpo.keys())
    # because sometime HPODisorderAssociation dict don't exist 
    if "HPODisorderAssociation" in section_hpo.keys():    
        for one_hpodict in section_hpo['HPODisorderAssociation']:
            hpo_id = one_hpodict['HPO']['HPOId']
            hpo_list.append(hpo_id)
            all_hpo_orphacode.append(hpo_id)
    orphacode_info[section_disorder['OrphaCode']] = set(hpo_list)

In [10]:
len(orphacode_info.keys())

4242

###  III. Patients - HPOs
#### Get Solved case confirmed

In [11]:
pd_phenopacket_confirmed = pd.read_excel(path_input + "/PATIENTS SOLVED_FC_v1.xlsx",engine='openpyxl',sheet_name='Feuil2')
pd_phenopacket_confirmed = pd_phenopacket_confirmed[pd_phenopacket_confirmed['Result'] == "yes"]
patients_c = pd_phenopacket_confirmed['Patient ID'].drop_duplicates().tolist()

print("{} patients solved confirmed".format(len(patients_c)))

143 patients solved confirmed


c:\Users\mchahdil\AppData\Local\miniconda3\envs\tuto_website\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [12]:
# path patients
files_json = os.listdir(path_phenopacket)
 
# remove the desktop.ini' and keep only phenopackets
patients_json = []
for op in files_json:
    # select only patients solved confirmed
    op_split= op.split('.')[0]
    if op_split in patients_c:
        patients_json.append(op)


In [13]:
len(patients_json)

142

In [14]:

patientsinfo = dict()
all_hpo_patients = []

pheno_with_invalid_id =set()
empty_no_phenotypic_feature = set()
for onep in patients_json:
    id_list = []
    with open(path_phenopacket+str("\\"+onep),'r',encoding = 'utf8') as file_phenopacket:
        one_result = json.load(file_phenopacket)

        one_phenopacket_result = one_result['phenopacket']
        id_phenopacket = one_phenopacket_result['id']
        # phenotypicFeatures dict key store hpo
        if 'phenotypicFeatures' in one_phenopacket_result.keys():
            #  list_hpo contains all HPO terms
            list_hpo = one_phenopacket_result['phenotypicFeatures']

            if not bool(list_hpo):
                # if this list is empty
                empty_no_phenotypic_feature.add(id_phenopacket)
            else :
                hpo_temp = []
                for onehpo in list_hpo:
                    if not bool(onehpo):
                        # sometime it s like this  [{}] ... list filled with en empty dict !
                        empty_no_phenotypic_feature.add(id_phenopacket)
                    else:
                        # fill the hpo_temp list
                        try:
                            if (("Invalid id" == onehpo['type.label'])):
                                pheno_with_invalid_id.add(id_phenopacket)
                                pass
                            if ( 'type.negated' not in onehpo.keys()):
                                hpo_temp.append(onehpo['type.id'])
                                all_hpo_patients.append(onehpo['type.id'])

                        except KeyError:
                            # json format dict may also have this structure : label and not type.label

                            if (("Invalid id" == onehpo['type']['label']) ):
                                pheno_with_invalid_id.add(id_phenopacket) 
                                pass
                            if ( 'negated' not in onehpo.keys()):
                                hpo_temp.append(onehpo['type']['id'])
                                all_hpo_patients.append(onehpo['type']['id'])


                if len(hpo_temp) >= 5:
                    if ((id_phenopacket not in pheno_with_invalid_id) and (id_phenopacket not in empty_no_phenotypic_feature)):
                        # if the number of element in hpo_temp is hight than 5 then the phenopacket contains more than 5HPO
                        patientsinfo[id_phenopacket] = hpo_temp


 

### IV Count HPOs per patient and HPOs per Orpha
Compte la fréquence d'apparition des HPO dans les patients puis dans les orphacode
pd deux dict

Count HPO per patient

In [15]:
patient_hpo_item = collections.Counter(all_hpo_patients)
len(patientsinfo)

142

In [16]:
#nb of different type HPO 
len(patient_hpo_item)

770

Count HPO per Orphacode

In [17]:
orphacode_hpo_item = collections.Counter(all_hpo_orphacode)

In [18]:
# nb of hpo tot sans doublons
tot_nb_items_hpo_orphacode = sum(list(orphacode_hpo_item.values()))

In [19]:
#nb of different type HPO 
print('nb hpo all : {}\tnb hpo unique : {}\tnb disease : {}'.format(tot_nb_items_hpo_orphacode,len(orphacode_hpo_item),len(orphacode_info)))

nb hpo all : 111765	nb hpo unique : 8305	nb disease : 4242


### V Build the df IC of all hpo in orphanet

In [20]:
# from collection dict type to list 
orphacode_hpo_list = list(orphacode_hpo_item.keys())
print('nb of HPO available in Orphanet : {}'.format(len(orphacode_hpo_list)))

nb of HPO available in Orphanet : 8305


Build dict for each hpo get theirs descants and counts occurend of those hpo in orphanet disease 
<br> (plus simple apres plus besoin de calculer a chaque fois on va utiliser ce dictionnaire plutot qui est calculer une fois )

In [21]:
dict_hpo_ic = {}
for onehpo in orphacode_hpo_list:
    try :
        all_LCA = extracts_LCAdescendant(onehpo,orphacode_hpo_item)
        #print("Le nb de LCA et ses descendants : {}".format(len(all_LCA)))

        orpha_LCAs = enumerated_LCAdescents_per_disease(all_LCA,orphacode_info)
        numerator_ic = len(orpha_LCAs)
        #print('Nb of diseases annotated with LCA and theirs descendant : {}'.format(numerator_ic) )

        ic = -math.log(len(orpha_LCAs) / len(orphacode_info))
        #print("IC : {}".format(ic))
        dict_hpo_ic[onehpo] = [ic,numerator_ic]
    except RuntimeError:
        print("{} Unknown HPO term".format(onehpo))

HP:0001578 Unknown HPO term
HP:0200115 Unknown HPO term
HP:4000141 Unknown HPO term


In [22]:
### verication with know ic (cf script  whhy_not_the_same.ipynb
print("This hpo HP:0007210 should be 4,52 : {}".format(dict_hpo_ic['HP:0007210']))
print("This hpo HP:0000924 should be 0,49 : {}".format(dict_hpo_ic['HP:0000924']))


This hpo HP:0007210 should be 4,52 : [4.524148738635533, 46]
This hpo HP:0000924 should be 0,49 : [0.4914483395246388, 2595]


### VI Resnik similarity for patient - orpha

- construction d'une boucle pour chaque patient et pour chacun de ses HPO on va les confrontée avec tout les HPO de la maladies donc pour le second HPO du patient on confront encore tout les HPO de la maladie ect... 
- donc pour chaque hpo patient vs tout les hpo maladie on regarde l'ancetre commun le plus proche 
- puis on calcul le IC donc resnik (qui est obtenu via le dict)
- pour chaque pair d'hpo patient -maladie on selectionne le LCA qui a un ic le plus haut  

- quand on construit la df resnik  on fait la somme de tout les resnik obtenu et on divise par le nombre de hpo du patient 

<br><br>
exemple : 
 
HPO patients                                                    LE HPO ORPHA qui a le max rensik                           resnik
Somatic sensory dysfunction (HP:0003474)                     Absent Achilles reflex (HP:0003438)                          0.38                                                        
Distal upper limb amyotrophy (HP:0007149)                    Intrinsic hand muscle atrophy (HP:0008954)                   4.66                                                        
Distal lower limb amyotrophy (HP:0008944)                    Intrinsic hand muscle atrophy (HP:0008954)                   3.73                                                        
Distal upper limb muscle weakness (HP:0008959)               Tibialis muscle weakness (HP:0008963)                        3.48                                                        
Proximal muscle weakness in lower limbs (HP:0008994)         Progressive proximal muscle weakness (HP:0009073)            3.51                                                        
Distal lower limb muscle weakness (HP:0009053)               Tibialis muscle weakness (HP:0008963)                        4.28                                                        
Progressive proximal muscle weakness (HP:0009073)            Progressive proximal muscle weakness (HP:0009073)            5.40                                                        
sum 25.44 div by 7 = 3.63 (returned value)
 

#### Get the LCA and its calculated the IC

In [23]:
all_interractions_hpo = set()
all_detail_interactions = []

hpo_not_use_in_orphacode = []
hpo_no_AC_common = []
hpo_unkown = []
for one_patient in patientsinfo.keys():
    #if one_patient == "P0001068" : 
        # get the HPO list of the patient of interest
        hpo_list_POF = patientsinfo[one_patient]
        #hpo_list_POF = ['HP:0040068'] # 0007149 0040068

        for one_orpha in orphacode_info.keys():
            #if i <=600: #i <=20 : #one_orpha == "324442" : 178400  
                
                # ge the HPO list of the orpha curently tested
                hpo_list_OOI = orphacode_info[one_orpha]
                #hpo_list_OOI = ['HP:0011842'] # 0003438 

                for onehpo_p in hpo_list_POF:
                    # dict to store the resnik sim of each hpo from orphacode BASED ON A SINGLE HPO PATIENT
                    hpo_ooi_LCA_resnik = {}
                    try :
                        ### get the type HPOTerm for hpo3 library
                        term_patient = Ontology.get_hpo_object(onehpo_p)

                        for onehpo_o in hpo_list_OOI:
                            """ on a besoin de toutes ces sim pour en selectionner un seul et faire la meme chose pour 
                                tout les hpo des patients """
                        
                            ### get the type HPOTerm for hpo3 library
                            term_disease = Ontology.get_hpo_object(onehpo_o)
                            
                            #print("########################")
                            #print("Patient : {}\t HPO patient:\t{}".format(one_patient,term_patient))
                            #print("Orphacode : ORPHA:{}\t HPO disease:\t{}".format(one_orpha,term_disease))

                            ### Identification des Ancêtres Communs : 
                            all_common_ancestors = term_patient.common_ancestors(term_disease)

                            # we have all the commons ancestor let get the closest which is the firt one in the set (converted in a list)
                            if not list(all_common_ancestors):
                                #print(" cas pas d'ancetre commun retrouver (situation impossible)")
                                #excluded 
                                hpo_no_AC_common.append((one_patient,str("ORPHA:"+one_orpha),term_patient.id,term_disease.id))

                            else: 
                                ### get the lca based on the list of commons ancestor
                                # contain the get_parent_of_ancestor function
                                list_LCAs = get_deepest_ancestor(all_common_ancestors) 
                                #print("LCAs : {}".format(list_LCAs))

                                for id_LCA in list_LCAs:
                                    try : 
                                        resnik = dict_hpo_ic[id_LCA][0]
                                        all_detail_interactions.append((one_patient,str("ORPHA:"+one_orpha),term_patient.id,term_disease.id,
                                                                                                        id_LCA,dict_hpo_ic[id_LCA][1],resnik))     
                                        # for each hpo FROM ORPHACODE and the hpo for patient get the LCA and store its resnik
                                        hpo_ooi_LCA_resnik[term_disease.id] = [resnik,id_LCA]


                                    except KeyError:
                                        # This HPO is not use in the Orphanet ontho
                                        hpo_not_use_in_orphacode.append(id_LCA)
                                        # set resnik the same as rsd set it 
                                        resnik = -math.log(1 / 4210)
                                        hpo_ooi_LCA_resnik[term_disease.id] = [resnik,id_LCA]

                                                                

                                    
                                    
                                # one the dict is build we extract the max resnik
                                max_resnik_disease_hpo = max(hpo_ooi_LCA_resnik, key=hpo_ooi_LCA_resnik.get)
                                max_resnik_disease_hpo_sim = hpo_ooi_LCA_resnik[max_resnik_disease_hpo][0]
                                max_resnik_disease_hpo_LCA = hpo_ooi_LCA_resnik[max_resnik_disease_hpo][1]

                                all_interractions_hpo.add((one_patient,str("ORPHA:"+one_orpha),term_patient.id,max_resnik_disease_hpo,max_resnik_disease_hpo_sim,max_resnik_disease_hpo_LCA))

                    except RuntimeError:
                        # unknow hpo terms
                        # un des deux 
                        hpo_unkown.append((term_disease.id,term_patient.id))

#df_hpo_no_AC_common = pd.DataFrame(hpo_no_AC_common, columns=["phenopacket",'orphacode','HP_patient','HP_disease'])
#detail_resnik_calcul = pd.DataFrame(all_detail_interactions, columns=["phenopacket",'orphacode','HP_patient','HP_disease','HP_LCA',"nb_times_LCA_in_orphanet",'IC_LCA'])

hpo_description = pd.DataFrame(all_interractions_hpo, columns=["phenopacket",'orphacode','HP_patient','HP_max_LCA','resnik','HP_LCA'])
# remove all row with HP:0000001 in the hpo_description
hpo_description_f= hpo_description[~hpo_description['HP_LCA'].str.contains("HP:0000001")]

##################
print("UNKNOWN  HPO {}".format(len(set(hpo_unkown))))
#print("No common_ancestors between HPO disease and HPO patient  {}".format(len(df_hpo_no_AC_common.drop_duplicates())))
print("HPO not in the Orphanet onthology   {}".format(len(set(hpo_not_use_in_orphacode))))


In [ ]:
#minidf = hpo_description_f[hpo_description_f["orphacode"] == "ORPHA:192" ]
#minidf.to_excel(path_output+"hpo_description.xlsx")

##### Try to undestands why hpo dont have ancestor in commons 

In [ ]:

# df_hpo_no_AC_common 

In [ ]:
## verif common ancestor
# term_patient = Ontology.get_hpo_object("HP:0003701")
# term_disease = Ontology.get_hpo_object("HP:0040073")

# term_patient.common_ancestors(term_disease)

set()

##### Calcul Resnik 

In [ ]:
list_df_pheno= hpo_description_f['phenopacket'].drop_duplicates().to_list()
list_df_orpha= hpo_description_f['orphacode'].drop_duplicates().to_list()

all_interaction_resnik = []
for onephenopacket in list_df_pheno:
    # split the df by patients
    mini_df= hpo_description_f[hpo_description_f['phenopacket']== onephenopacket]
    
    for oneorphacode in list_df_orpha:
        # split by orpha
        mini_mini_df= mini_df[mini_df['orphacode']==oneorphacode]
        # get unique LCA from this interactions 
        all_hpoLCA = mini_mini_df['HP_LCA'].drop_duplicates()
        # smerge all ic and divided by the number of LCA found for this orpha
        sum_ic = 0
        for oneLCA in all_hpoLCA:
            sum_ic = sum_ic + dict_hpo_ic[oneLCA][0]
        final_ic = sum_ic / len(all_hpoLCA)

        all_interaction_resnik.append((onephenopacket,oneorphacode,final_ic))
 
df_resnik = pd.DataFrame(all_interaction_resnik, columns=["phenopacket",'ORPHAcode','resnik'])
 

etablir le rank cad pour chaque maladie garder uniquement le top 50

In [ ]:
# construiction a partir de la df resnik d'un dict de df 
list_df_pheno= df_resnik['phenopacket'].drop_duplicates().to_list()
list_df_orpha= df_resnik['ORPHAcode'].drop_duplicates().to_list()

all_interaction_resnik = []
dict_of_resnik_df = {}

for onephenopacket in list_df_pheno:
    # split the df by patients
    mini_df= df_resnik[df_resnik['phenopacket']== onephenopacket]
    # sort by score
    df_op = mini_df.sort_values('resnik',ascending=False)
    # keep the top 50
    df_op = df_op.head(50)
    df_op.reset_index(drop=True)
    dict_of_resnik_df[onephenopacket] = df_op

    df_op= pd.DataFrame([], columns=["phenopacket",'ORPHAcode','resnik'])


dict_of_resnik_df

{'P0001068':      phenopacket     ORPHAcode    resnik
 2255    P0001068   ORPHA:98856  3.002744
 3578    P0001068   ORPHA:98897  2.991674
 1681    P0001068   ORPHA:63273  2.925893
 3047    P0001068  ORPHA:101097  2.901952
 1613    P0001068  ORPHA:178400  2.897214
 1242    P0001068   ORPHA:98912  2.894281
 2826    P0001068     ORPHA:610  2.862502
 2190    P0001068   ORPHA:99953  2.860372
 257     P0001068  ORPHA:435387  2.859376
 1896    P0001068  ORPHA:466768  2.811051
 2875    P0001068  ORPHA:399086  2.809856
 177     P0001068  ORPHA:101077  2.798716
 281     P0001068  ORPHA:254361  2.780477
 2813    P0001068   ORPHA:99956  2.751191
 1444    P0001068   ORPHA:99947  2.732660
 82      P0001068  ORPHA:399103  2.732213
 28      P0001068  ORPHA:276435  2.729070
 392     P0001068  ORPHA:482601  2.719368
 904     P0001068  ORPHA:457050  2.684322
 3134    P0001068  ORPHA:497764  2.674840
 2055    P0001068   ORPHA:90103  2.672626
 56      P0001068  ORPHA:101076  2.667706
 1991    P0001068  ORP

In [ ]:
all_interractions=[]


for key,dfval in dict_of_resnik_df.items():
    #print("set the df for the patient {}".format(key))
    dict_df_one_P_hpo3 = dfval.to_dict('index')

    i = 0 # for the rank
    for value in dict_df_one_P_hpo3.values():
        hpo3_onepheno = value['phenopacket']
        hpo3_oneorpha= value['ORPHAcode']
        hpo3_onescore= value['resnik']

        i = i + 1
        
        all_interractions.append((hpo3_onepheno,hpo3_oneorpha,hpo3_onescore,i))



df_resnik_with_rank = pd.DataFrame(all_interractions, columns=["phenopacket",'ORPHAcode','resnik','rank'])
 

,phenopacket,ORPHAcode,resnik,rank
0,P0001068,ORPHA:98856,3.002744,1
1,P0001068,ORPHA:98897,2.991674,2
2,P0001068,ORPHA:63273,2.925893,3
3,P0001068,ORPHA:101097,2.901952,4
4,P0001068,ORPHA:178400,2.897214,5
5,P0001068,ORPHA:98912,2.894281,6
6,P0001068,ORPHA:610,2.862502,7
7,P0001068,ORPHA:99953,2.860372,8
8,P0001068,ORPHA:435387,2.859376,9
9,P0001068,ORPHA:466768,2.811051,10


### Export

In [ ]:
# 0 = own formula, 3 own formula with IC hpo3
# 1 = own CUM  formula, 4 own  CUM formula with IC hpo3

'\noneIC = detail_resnik_calcul2[detail_resnik_calcul2[\'HP_patient\'] == "HP:0008959"][\'IC_ancestor\'][1]\nprint(oneIC)\n#10puissance (-IC)\nprint(10**(-oneIC))\n'

In [ ]:
patient_OI = "P0001068"

In [ ]:

df_resnik_with_rank.to_excel(path_output+"df_resnik_with_rank.xlsx") 

In [ ]:
df_hpo_no_AC_common= df_hpo_no_AC_common.drop_duplicates()
df_hpo_no_AC_common.to_excel(path_output+str(patient_OI)+"_resnik_no_common_ancestor.xlsx")


In [ ]:
export_file(path_output, list(set(hpo_unkown)),str(patient_OI)+"_resnik_unknowHPO")


In [ ]:
export_file(path_output, list(set(hpo_not_use_in_orphacode)),str(patient_OI)+"_resnik_not_in_orphanet_HPO")


In [ ]:
Apporter leur exp dans les taches que le doctorant doit effectué, corriger si besoin, laisser le doctorant suivre ses pistes (souvent pas bonne)
Avancement dans les problematiques en lien avec la these, recherche biblo, se poser les bonne questions,
Produire une cohérence dans le travail et les taches effectué
